This will track of performance tests and enhancements to code provided by 10x Genomics.

The code below was originally provided by 10x Genomics python tutorial for their 1.3 million cell dataset.
http://cf.10xgenomics.com/supp/cell-exp/megacell_tutorial-1.0.1.html

Working directory: /local/scratch/dolley/hdf5
Minimum 5GB RAM: qlogin -P owhite-nemo -q interactive.q -l mem_free=5G

In [4]:
#!/usr/bin/env python3

#import modules, define some functions for loading, saving and processing a gene-barcode matrix
#%matplotlib inline
import collections
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp_sparse
import tables

np.random.seed(0)

GeneBCMatrix = collections.namedtuple('GeneBCMatrix', ['gene_ids', 'gene_names', 'barcodes', 'matrix'])

def get_matrix_from_h5(filename, genome):
    with tables.open_file(filename, 'r') as f:
        try:
            dsets = {}
            for node in f.walk_nodes('/' + genome, 'Array'):
                dsets[node.name] = node.read()
            matrix = sp_sparse.csc_matrix((dsets['data'], dsets['indices'], dsets['indptr']), shape=dsets['shape'])
            return GeneBCMatrix(dsets['genes'], dsets['gene_names'], dsets['barcodes'], matrix)
        except tables.NoSuchNodeError:
            raise Exception("Genome %s does not exist in this file." % genome)
        except KeyError:
            raise Exception("File is missing one or more required datasets.")

def get_expression(gbm, gene_name):
    gene_indices = np.where(gbm.gene_names == gene_name)[0]
    if len(gene_indices) == 0:
        raise Exception("%s was not found in list of gene names." % gene_name)
    return gbm.matrix[gene_indices[0], :].toarray().squeeze()


def main():
  filename = '/local/scratch/dolley/hdf5/1M_neurons_filtered_gene_bc_matrices_h5.h5'
  genome = 'mm10'
  #gene_symbol = ['Stmn2', 'Tbr1', 'Olig1']
  gene_symbol = ['Stmn2']

  matrix = get_matrix_from_h5(filename, genome)
  print("HDF5 file loaded. Matrix created")

  #print("example gene_symbol: ", matrix.gene_names[10])

  expression = []
  for gene in gene_symbol:
      values = get_expression(matrix, gene.encode())
      expression.append({ 'gene_sym': gene, 'raw_values': values })

  print('1st expression entry: ', expression[0]['gene_sym'], ' ', expression[0]['raw_values'][0])
  print("\nDone!\n")


cProfile.run('main()', '/local/scratch/dolley/hdf5/tutorial_script.profile.dat')

 The above cell was run on the grid with 5GB RAM. cProfile was used to capture performances.

Running command: `cprofilev -f tutorial_script.profile.dat` opens browser to the profile:
   cprofilev: https://github.com/ymichael/cprofilev

Profile is sorted by 'tottime'

In [ ]:
Fri Jan 26 15:57:49 2018    tutorial_script.profile.dat

         2634 function calls (2565 primitive calls) in 102.402 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7   81.839   11.691   81.839   11.691 {method '_read_array' of 'tables.hdf5extension.Array' objects}
       20   12.332    0.617   12.332    0.617 {method 'reduce' of 'numpy.ufunc' objects}
        1    6.198    6.198    6.198    6.198 {built-in method scipy.sparse._sparsetools.get_csr_submatrix}
        1    1.527    1.527  102.402  102.402 <string>:1(<module>)
        7    0.467    0.067    0.467    0.067 {method '_g_get_lchild_attr' of 'tables.hdf5extension.Group' objects}
        7    0.010    0.001    0.013    0.002 {method '_open_array' of 'tables.hdf5extension.Array' objects}
        1    0.007    0.007    0.007    0.007 {built-in method scipy.sparse._sparsetools.csc_tocsr}
       43    0.002    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numeric.py:424(asarray)
        7    0.002    0.000    0.018    0.003 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/carray.py:133(__init__)
       65    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.array}
        8    0.001    0.000    0.016    0.002 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:199(__init__)
        7    0.001    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:143(_normalize_default)
        7    0.001    0.000    0.003    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:427(from_kind)
        1    0.001    0.001    0.001    0.001 {built-in method scipy.sparse._sparsetools.csr_todense}
        2    0.001    0.000    0.001    0.000 {method 'astype' of 'numpy.ndarray' objects}
        2    0.001    0.000    0.001    0.000 {built-in method posix.access}
        1    0.000    0.000    0.001    0.001 {method '_g_new' of 'tables.hdf5extension.File' objects}
       36    0.000    0.000    0.000    0.000 {method '_g_getattr' of 'tables.hdf5extension.AttributeSet' objects}
       43    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:286(__getattr__)
       15    0.000    0.000    0.000    0.000 {method '_g_get_objinfo' of 'tables.hdf5extension.Group' objects}
        1    0.000    0.000    0.000    0.000 {method '_close_file' of 'tables.hdf5extension.File' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.zeros}
        1    0.000    0.000   12.346   12.346 ./tutorial_script.py:31(get_expression)
       11    0.000    0.000   12.333    1.121 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:119(get_index_dtype)
        9    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:205(__init__)
        7    0.000    0.000    0.000    0.000 {method '_g_close' of 'tables.hdf5extension.Leaf' objects}
       23    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/getlimits.py:507(__init__)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:128(check_format)
       45    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:383(register_node)
        5    0.000    0.000   12.335    2.467 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:25(__init__)
       61    0.000    0.000    0.000    0.000 {method 'pop' of 'tables.lrucacheextension.NodeCache' objects}
      186    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000   88.529   88.529 ./tutorial_script.py:18(get_matrix_from_h5)
        7    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:517(__init__)
        1    0.000    0.000    0.000    0.000 {tables.utilsextension.read_f_attr}
        7    0.000    0.000   81.842   11.692 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/array.py:859(read)
        7    0.000    0.000   81.839   11.691 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/array.py:829(_read)
        8    0.000    0.000    0.486    0.061 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:1148(_g_load_child)
       53    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:145(__contains__)
       47    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
     36/9    0.000    0.000    0.486    0.054 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:409(get_node)
      5/1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/abc.py:194(__subclasscheck__)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:120(_normalize_shape)
        7    0.000    0.000    0.467    0.067 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:301(_g_get_child_leaf_class)
        1    0.000    0.000    0.000    0.000 {method '_g_list_group' of 'tables.hdf5extension.Group' objects}
       44    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:396(cache_node)
       46    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:820(__setattr__)
       10    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:470(_f_close)
        8    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:348(_g_set_location)
       10    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.empty}
       10    0.000    0.000    0.000    0.000 {method 'clear' of 'dict' objects}
        8    0.000    0.000    0.487    0.061 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:714(_f_iter_nodes)
        1    0.000    0.000    0.008    0.008 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csc.py:137(tocsr)
        1    0.000    0.000    6.204    6.204 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csr.py:433(_get_submatrix)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/path.py:172(join_path)
        7    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:309(_g_post_init_hook)
        7    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/flavor.py:380(_conv_numpy_to_numpy)
        7    0.000    0.000    0.486    0.069 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:684(_f_get_child)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:1058(prune)
        6    0.000    0.000    6.161    1.027 {method 'min' of 'numpy.ndarray' objects}
       63    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:64(issysattrname)
    46/19    0.000    0.000    0.486    0.026 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:1537(_get_node)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:310(__init__)
        1    0.000    0.000    0.000    0.000 {method '_g_get_gchild_attr' of 'tables.hdf5extension.Group' objects}
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:78(set_shape)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:155(__setitem__)
        1    0.000    0.000  102.402  102.402 {built-in method builtins.exec}
       36    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:197(_g_getnode)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/array.py:112(rowsize)
        9    0.000    0.000    0.000    0.000 {method '_g_list_attr' of 'tables.hdf5extension.AttributeSet' objects}
        1    0.000    0.000    0.002    0.002 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:748(__init__)
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2654(close)
        6    0.000    0.000    6.171    1.029 {method 'max' of 'numpy.ndarray' objects}
    30/21    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/path.py:203(split_path)
        7    0.000    0.000    0.013    0.002 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/array.py:232(_g_open)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:200(isshape)
        7    0.000    0.000    0.486    0.069 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/misc/proxydict.py:35(__getitem__)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:430(_g_del_location)
        8    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2456(prod)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:265(_unpack_index)
       26    0.000    0.000    0.000    0.000 {method 'match' of '_sre.SRE_Pattern' objects}
       25    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:336(_g_check_open)
       89    0.000    0.000    0.000    0.000 {method 'startswith' of 'str' objects}
        7    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/flavor.py:113(array_of_flavor2)
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:199(keys)
        8    0.000    0.000    0.487    0.061 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2028(walk_nodes)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:340(_g_add_children_names)
        1    0.000    0.000   12.336   12.336 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csc.py:160(__getitem__)
        4    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:500(_close_nodes)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/filters.py:231(_unpack)
       35    0.000    0.000    0.000    0.000 {method 'decode' of 'bytes' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csr.py:131(transpose)
       12    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:631(issubclass_)
        1    0.000    0.000  100.875  100.875 ./tutorial_script.py:37(main)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:762(<listcomp>)
        1    0.000    0.000    6.205    6.205 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csr.py:236(__getitem__)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:400(_process_range)
       10    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:236(pop)
        7    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/flavor.py:368(conv_to_numpy)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:370(_calc_nrowsinbuf)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:763(_f_walk_groups)
       15    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:387(_g_check_has_child)
        2    0.000    0.000    0.000    0.000 {method '_g_close_group' of 'tables.hdf5extension.Group' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:101(__init__)
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:531(close_subtree)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:71(__init__)
       12    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
       17    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/getlimits.py:532(max)
        6    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/getlimits.py:518(min)
       20    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/utils.py:238(newfget)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:20(upcast)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:305(__new__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:416(_process_range_read)
        6    0.000    0.000    6.161    1.027 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/_methods.py:28(_amin)
        9    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:165(_v_attrs)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:729(_f_close)
        1    0.000    0.000    0.009    0.009 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:940(toarray)
        7    0.000    0.000    0.016    0.002 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:241(__init__)
        6    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:699(issubdtype)
       13    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:58(__iter__)
        7    0.000    0.000    0.000    0.000 {method '_g_new' of 'tables.hdf5extension.Leaf' objects}
        2    0.000    0.000    0.000    0.000 {method '_g_open' of 'tables.hdf5extension.Group' objects}
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/data.py:22(__init__)
        6    0.000    0.000    6.171    1.029 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/_methods.py:25(_amax)
        3    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:635(__init__)
        7    0.000    0.000    0.486    0.069 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:40(_get_value_from_container)
       27    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
       23    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:100(getnnz)
       72    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:130(__getitem__)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:365(__init__)
       36    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:200(_v_node)
        2    0.000    0.000    0.000    0.000 {built-in method posix.stat}
       23    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:213(nnz)
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:882(_g_close)
        8    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:833(_g_check_group)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:837(__get_root_group)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:1106(__init__)
        9    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:26(__exit__)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:36(__init__)
        7    0.000    0.000    0.003    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/flavor.py:158(internal_to_flavor)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:91(to_native)
        2    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/earray.py:137(__init__)
       21    0.000    0.000    0.000    0.000 {method 'endswith' of 'bytes' objects}
        6    0.000    0.000    0.000    0.000 {built-in method builtins.max}
    80/78    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        5    0.000    0.000    0.000    0.000 {method 'newbyteorder' of 'numpy.dtype' objects}
       14    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/array.py:104(nrows)
       10    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/_lib/_util.py:128(_prune_array)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.where}
       28    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:193(maindim)
        7    0.000    0.000    0.002    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/flavor.py:136(flavor_to_flavor)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:20(__enter__)
        4    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:694(__init__)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/misc/proxydict.py:81(_get_container)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:1548(get_node)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:962(find_common_type)
        9    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
        1    0.000    0.000    6.131    6.131 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csc.py:111(transpose)
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numeric.py:1917(isscalar)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/abc.py:178(__instancecheck__)
        8    0.000    0.000    0.000    0.000 {method 'indices' of 'slice' objects}
        7    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
       16    0.000    0.000    0.000    0.000 {method 'count' of 'str' objects}
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/filters.py:314(__init__)
       10    0.000    0.000    0.000    0.000 {method 'endswith' of 'str' objects}
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:16(__init__)
        6    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:70(__contains__)
       10    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x2b39494cb300}
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/utils.py:41(correct_byteorder)
       14    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/misc/proxydict.py:42(__setitem__)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csr.py:436(process_slice)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:540(<listcomp>)
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:183(isscalarlike)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/six.py:580(iterkeys)
       54    0.000    0.000    0.000    0.000 {method 'isupper' of 'str' objects}
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/utils.py:139(check_file_access)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:560(shutdown)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:1119(_process_toarray_args)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:331(_check_boolean)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:205(flavor)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:293(_check_ellipsis)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:497(size)
        8    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:188(isintlike)
        2    0.000    0.000    6.132    3.066 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:633(__getattr__)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:284(_g_get_child_group_class)
        8    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:68(<listcomp>)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/six.py:586(iteritems)
        2    0.000    0.000    0.001    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:237(_g_post_init_hook)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/genericpath.py:27(isfile)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:204(__init__)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:448(drop_from_cache)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:140(<genexpr>)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/leaf.py:144(name)
       14    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:875(_g_close_descendents)
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2711(__exit__)
        1    0.000    0.000    0.002    0.002 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:224(open_file)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:267(__del__)
        1    0.000    0.000    0.001    0.001 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:893(_f_close)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:115(remove)
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:52(_commit_removals)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:111(add)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:288(__del__)
        8    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/_methods.py:34(_prod)
       37    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
       17    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:101(get_shape)
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/base.py:1202(isspmatrix)
       10    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:123(get_handlers_by_name)
        6    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/misc/proxydict.py:25(__init__)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:142(__len__)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:453(_g_post_init_hook)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/compressed.py:1033(__set_sorted)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:939(_can_coerce_all)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2076(walk_groups)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/path.py:227(isvisiblename)
        5    0.000    0.000    0.000    0.000 {method '__subclasses__' of 'type' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/group.py:809(__getattr__)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2122(_check_group)
        3    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/data.py:25(_get_dtype)
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csc.py:224(_swap)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/registry.py:70(get_class_by_name)
        1    0.000    0.000    0.000    0.000 {method 'squeeze' of 'numpy.ndarray' objects}
        5    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/_weakrefset.py:81(add)
        1    0.000    0.000    0.000    0.000 <string>:12(__new__)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:1013(<listcomp>)
        6    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.can_cast}
        5    0.000    0.000    0.000    0.000 {method '__subclasshook__' of 'object' objects}
        9    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/attributeset.py:660(_g_close)
        2    0.000    0.000    0.000    0.000 {method '_g_new' of 'tables.hdf5extension.Node' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:261(update)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:765(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       10    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/csr.py:231(_swap)
        2    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numexpr/utils.py:61(set_vml_num_threads)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/genericpath.py:16(exists)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:546(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:557(<listcomp>)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:551(<listcomp>)
        9    0.000    0.000    0.000    0.000 {method 'remove' of 'set' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:1966(iter_nodes)
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2094(_check_open)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
        1    0.000    0.000    0.000    0.000 {method '_get_file_id' of 'tables.hdf5extension.File' objects}
        7    0.000    0.000    0.000    0.000 {method 'insert' of 'list' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/scipy/sparse/sputils.py:227(isdense)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.iter}
        2    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/weakref.py:122(_commit_removals)
        9    0.000    0.000    0.000    0.000 {method '_g_new' of 'tables.hdf5extension.AttributeSet' objects}
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:769(<listcomp>)
        4    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'update' of 'dict' objects}
        7    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/atom.py:526(<genexpr>)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/numpy/core/numerictypes.py:1014(<listcomp>)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/file.py:2706(__enter__)
        1    0.000    0.000    0.000    0.000 /usr/local/packages/python-3.5.2/lib/python3.5/site-packages/tables/node.py:156(_g_getparent)
        1    0.000    0.000    0.000    0.000 {built-in method _stat.S_ISREG}
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'list' objects}

